# Abro mi dataset y lo preparo para la función

## Abro el archivo json

In [7]:
import pandas as pd

# Especifica la ruta del archivo JSON
ruta_archivo_json = "/content/drive/MyDrive/Ayuda_fastAPI_y_RENDER/01_Dataset/df_usuarios.json"  # Cambia esta línea con la ruta del archivo JSON

# Lee el archivo JSON y carga los datos en un DataFrame
df_usuarios = pd.read_json(ruta_archivo_json, orient='index')

df_usuarios.sample(10)

,Usuario,user_ID,Estatura (cm),Edad,Opinion,Gastos,Tipo de Sangre,Año
8842,kmorrow,9724,189.64,31,negativa,1321.71,A+,1989
5773,uthomas,5784,179.30,36,negativa,1324.05,A-,1989
3208,bethanysmith,4722,165.00,41,negativa,2378.16,B+,1991
4402,george34,9339,177.49,28,positiva,631.91,A-,1989
4440,jhoward,4398,178.69,49,negativa,4761.87,AB+,1985
1567,prodriguez,4432,152.94,60,negativa,626.69,B-,1990
5418,michelleparker,5368,168.76,41,negativa,606.93,A+,1990
7291,carriemeyer,6260,161.26,59,negativa,3314.93,A+,1990
1676,hgill,7471,169.27,27,negativa,1468.53,O-,1993
6553,shawalison,8529,164.43,40,negativa,4690.92,AB-,1994


## Borramos las columnas tipo de sanfre y opinion ya que no la vamos a necesitar para nuestra funcion

In [15]:
# Borrar dos columnas (por ejemplo, Columna1 y Columna2)
columnas_a_borrar = ['Tipo de Sangre', 'Opinion']
df_usuarios = df_usuarios.drop(columnas_a_borrar, axis=1)

## Vemos como queda nustro dataframe

In [16]:
df_usuarios.sample(10)

,Usuario,user_ID,Estatura (cm),Edad,Gastos,Año
5295,tony98,7678,162.15,57,1220.46,1989
6942,andersonhayden,4845,160.22,43,1993.47,1985
4394,gabrielgarza,2909,178.11,27,637.26,1985
4657,beth04,8008,155.66,30,1083.29,1986
8299,adamscarrie,4872,169.93,61,4086.55,1985
6951,baileyjennifer,4083,156.98,58,2955.80,1993
7165,mmeyer,6755,163.86,28,4435.64,1987
5857,ycox,9347,187.55,34,3548.20,1987
706,fsimmons,7677,151.28,63,3411.03,1992
8015,jhernandez,5939,180.03,23,881.58,1994


## Guardamos nuestro dataframe como archivo parquet con el nombre para el endpoint N°1

In [17]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Especificar la ruta completa del archivo Parquet
ruta_archivo_parquet = '/content/drive/MyDrive/Ayuda_fastAPI_y_RENDER/01_Dataset/dataset_endpoint_1.parquet'

# Convertir el DataFrame a un objeto Table de pyarrow
tabla_pyarrow = pa.Table.from_pandas(df_usuarios)

# Guardar el archivo Parquet en la ubicación especificada
pq.write_table(tabla_pyarrow, ruta_archivo_parquet)

print(f"DataFrame guardado como archivo Parquet en: {ruta_archivo_parquet}")

DataFrame guardado como archivo Parquet en: /content/drive/MyDrive/Ayuda_fastAPI_y_RENDER/01_Dataset/dataset_endpoint_1.parquet


# Funcion N°1 - ENDPOINT N°1

Creamos la funcion en base a la consigna dada, en este ejemplo:

Se nos pide hacer una funcion que al ingresar el año me devuelva el top 3 de users que mas gastaron ese año.

Ejemplo:

{'Año': 1989, 'Top 3 Gastadores':
[{'Usuario': 'brandon10', 'Gastos': 4999.05},
{'Usuario': 'shawnnewman', 'Gastos': 4998.64},
{'Usuario': 'willisjohn', 'Gastos': 4994.37}]}





In [14]:
import pandas as pd

def top_gastadores_por_anio(df, year):
    # Filtrar el DataFrame por el año proporcionado
    df_usuarios_filtered = df[df['Año'] == year]

    # Obtener el top 3 de usuarios que más gastaron en ese año
    top_gastadores = df_usuarios_filtered.groupby('Usuario')['Gastos'].sum().nlargest(3)

    # Crear el resultado en formato JSON
    result = {
        'Año': year,
        'Top 3 Gastadores': top_gastadores.reset_index().to_dict(orient='records')
    }
    return result

# Ejemplo de uso
result = top_gastadores_por_anio(df_usuarios, 1989)
print(result)

{'Año': 1989, 'Top 3 Gastadores': [{'Usuario': 'brandon10', 'Gastos': 4999.05}, {'Usuario': 'shawnnewman', 'Gastos': 4998.64}, {'Usuario': 'willisjohn', 'Gastos': 4994.37}]}


Transformo la Funcion N°1 en una funcion para fastAPI

In [ ]:
from fastapi import FastAPI, HTTPException
import pandas as pd
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder

app = FastAPI()

# Función para obtener el top de gastadores por año
def top_gastadores_por_anio(df, year):
    # Filtrar el DataFrame por el año proporcionado
    df_usuarios_filtered = df[df['Año'] == year]

    # Obtener el top 3 de usuarios que más gastaron en ese año
    top_gastadores = df_usuarios_filtered.groupby('Usuario')['Gastos'].sum().nlargest(3)

    # Crear el resultado en formato JSON
    result = {
        'Año': year,
        'Top 3 Gastadores': top_gastadores.reset_index().to_dict(orient='records')
    }
    return result

# Ruta para obtener el top de gastadores por año desde el archivo Parquet
@app.get("/top_gastadores/{year}")
async def top_gastadores_by_year(year: int):
    try:
        # Cambia la ruta del archivo según la ubicación de tu archivo df_endpoint_1.parquet
        df = pd.read_parquet("df_endpoint_1.parquet")

        # Aplicar la función para obtener el top de gastadores por año
        result = top_gastadores_por_anio(df, year)

        return JSONResponse(content=jsonable_encoder(result), media_type="application/json")
    except FileNotFoundError:
        raise HTTPException(status_code=404, detail="Archivo Parquet no encontrado")
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error al leer el archivo Parquet: {str(e)}")
